In [1]:
pip install pandas python-docx

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
import re
import random
from datetime import datetime

# Rutas
base_path = os.path.join('D:', 'ExamenParcial2ComputacionParaBioingenieria', 'parcial2_Machicado_Martinez_GabrielAntonio_ejercicio2')
csv_path = os.path.join(base_path, 'BBBP.csv')
output_path = os.path.join(base_path, 'informes_BBBP')
plantilla_path = os.path.join(base_path, 'plantilla_bbbp.docx')

os.makedirs(output_path, exist_ok=True)

# Leer CSV
try:
    df = pd.read_csv(csv_path)
    df = df.head(300)
except Exception as e:
    print(f"Error al leer el archivo CSV: {str(e)}")
    exit()

# Limpiar nombre de archivo
def clean_filename(name):
    if not isinstance(name, str):
        name = str(name)
    name = re.sub(r'[\\/*?:"<>|]', "", name)
    name = name.replace(' ', '_').replace(',', '_').replace('-', '_').replace('.', '_')
    return name[:50]

# Estilos personalizados

def set_title_style(paragraph):
    run = paragraph.runs[0]
    run.font.name = 'Showcard Gothic'
    run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')
    run.font.size = Pt(25)
    run.font.color.rgb = RGBColor(0x44, 0x72, 0xC4)
    run.font.bold = True
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

def set_heading_style(paragraph, level=2):
    run = paragraph.runs[0]
    run.font.name = 'Arial'
    run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')
    run.font.size = Pt(14 if level == 2 else 12)
    run.font.color.rgb = RGBColor(0x2B, 0x2B, 0x2B)
    run.font.bold = True
    paragraph.paragraph_format.space_after = Pt(6)

def set_body_style(paragraph):
    run = paragraph.runs[0]
    run.font.name = 'Arial'
    run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Segoe UI Black')
    run.font.size = Pt(11)
    paragraph.paragraph_format.space_after = Pt(12)
    paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

def add_divider(doc):
    p = doc.add_paragraph()
    run = p.add_run("―"*50)
    run.font.color.rgb = RGBColor(0xCC, 0xCC, 0xCC)
    p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

def add_bullet_list(doc, items):
    for item in items:
        p = doc.add_paragraph()
        run = p.add_run(f"• {item}")
        run.font.name = 'Calibri'
        run.font.size = Pt(11)
        p.paragraph_format.left_indent = Pt(18)
        p.paragraph_format.space_after = Pt(6)

# Texto introductorio
introduccion = """
El estudio de la permeabilidad de la barrera hematoencefálica (Blood-Brain Barrier Penetration, BBBP) es fundamental en el desarrollo de fármacos neuroactivos, ya que determina la capacidad de un compuesto para alcanzar su diana terapéutica en el sistema nervioso central (SNC). La barrera hematoencefálica (BHE) es una estructura altamente selectiva que protege al cerebro de sustancias potencialmente nocivas, pero también limita el paso de compuestos terapéuticos. Por ello, predecir y evaluar la capacidad de penetración de moléculas a través de la BHE es un paso crítico en el diseño de fármacos para enfermedades neurológicas y psiquiátricas.

En este informe se analizan diferentes compuestos químicos con potencial actividad farmacológica en el SNC, evaluando su probabilidad de penetración en la BHE (clasificada como "P" para penetrantes y "NP" para no penetrantes). Cada compuesto se describe mediante su estructura química en formato SMILES (Simplified Molecular Input Line Entry System), lo que permite una representación estandarizada para su análisis computacional. Además, se incluyen características fisicoquímicas relevantes, como peso molecular, coeficiente de partición octanol-agua (LogP), y la presencia de grupos funcionales donores o aceptores de enlaces de hidrógeno, parámetros clave en las reglas de filtrado para permeabilidad.

Este documento, elaborado en el marco del Examen Parcial 2 de Computación para Bioingeniería, tiene como objetivo proporcionar un formato estandarizado para la evaluación sistemática de compuestos con potencial aplicación en neurofarmacología. La información aquí recopilada servirá como base para futuros estudios in vitro e in silico, optimizando el proceso de descubrimiento de fármacos y reduciendo costos en etapas tempranas de investigación.
"""

# Descripciones

descriptions = {
    'p_np': {
        1: "Este compuesto ha demostrado capacidad para atravesar la barrera hematoencefálica (BBB+).",
        0: "Este compuesto no atraviesa eficientemente la barrera hematoencefálica (BBB-)."
    },
    'general': [
        "Este compuesto muestra propiedades farmacológicas interesantes.",
        "La estructura molecular sugiere potencial actividad en el SNC.",
        "Indicaciones de aplicación en trastornos neurodegenerativos.",
        "Características estructurales prometedoras para enfermedades cerebrales.",
        "Candidato para nuevas terapias neuroactivas."
    ],
    'smiles': [
        "La representación SMILES proporciona información sobre la estructura molecular.",
        "El formato SMILES permite visualizar conectividad y grupos funcionales.",
        "Notación lineal legible por máquina.",
        "El análisis del SMILES revela claves de permeabilidad BBB."
    ],
    'applications': [
        "Aplicación en tratamiento de Alzheimer o Parkinson.",
        "Uso potencial en desórdenes del SNC.",
        "Candidato a desarrollo de analgésicos centrales.",
        "Uso en trastornos psiquiátricos.",
        "Terapia contra tumores cerebrales."
    ]
}

success_count = 0

for i, row in df.iterrows():
    try:
        doc = Document(plantilla_path)

        # Encabezado
        header = doc.sections[0].header
        p = header.paragraphs[0] if header.paragraphs else header.add_paragraph()
        run = p.add_run(f"Reporte BBBP - {datetime.now().strftime('%d/%m/%Y')}")
        run.font.name = 'Arial'
        run.font.size = Pt(9)
        run.font.color.rgb = RGBColor(0x66, 0x66, 0x66)
        p.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

        # Título
        p = doc.add_paragraph("INFORME BBBP")
        set_title_style(p)

        # Autor
        p = doc.add_paragraph("Autor: Ing. Gabriel Machicado")
        p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        p.runs[0].font.color.rgb = RGBColor(0x44, 0x72, 0xC4)

        # Institución y fecha
        doc.add_paragraph("BioTech Bolivia").alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        doc.add_paragraph(f"Fecha: {datetime.now().strftime('%d de %B de %Y')}").alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Introducción
        p = doc.add_paragraph("\nINTRODUCCIÓN")
        set_heading_style(p)
        for paragraph in introduccion.strip().split('\n\n'):
            p = doc.add_paragraph(paragraph.strip())
            set_body_style(p)

        add_divider(doc)

        # Informe del compuesto
        p = doc.add_paragraph(f"Informe Detallado del Compuesto #{i+1}")
        set_title_style(p)

        compound_name = str(row['name']) if pd.notna(row['name']) else f"Compuesto_{i+1}"
        p = doc.add_paragraph(f"Nombre: {compound_name}")
        p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        p.runs[0].font.color.rgb = RGBColor(0x44, 0x72, 0xC4)

        # Resumen
        p = doc.add_paragraph("Resumen del Compuesto")
        set_heading_style(p)
        p_np_value = int(row['p_np']) if pd.notna(row['p_np']) else 0
        p = doc.add_paragraph(f"Estado BBB: {descriptions['p_np'].get(p_np_value)}")
        p.runs[0].font.bold = True
        doc.add_paragraph(random.choice(descriptions['general']))

        # Aplicación
        p = doc.add_paragraph("Aplicación Potencial")
        set_heading_style(p, level=3)
        add_bullet_list(doc, [random.choice(descriptions['applications'])])

        add_divider(doc)

        # Tabla
        p = doc.add_paragraph("Datos Técnicos")
        set_heading_style(p)
        table = doc.add_table(rows=1, cols=2)
        table.rows[0].cells[0].text = 'Campo'
        table.rows[0].cells[1].text = 'Valor'
        for col in ['num', 'p_np', 'smiles']:
            if col in row and pd.notna(row[col]):
                cells = table.add_row().cells
                cells[0].text = col.upper()
                cells[1].text = str(row[col])

        # Análisis SMILES
        if pd.notna(row['smiles']):
            p = doc.add_paragraph("Análisis SMILES")
            set_heading_style(p, level=3)
            add_bullet_list(doc, descriptions['smiles'])

        add_divider(doc)

        # Notas
        p = doc.add_paragraph("Notas Adicionales")
        set_heading_style(p)
        doc.add_paragraph("Este informe ha sido generado automáticamente por el sistema BBBP.")
        doc.add_paragraph(f"Fecha de generación: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

        # Pie de página
        footer = doc.sections[0].footer
        p = footer.paragraphs[0] if footer.paragraphs else footer.add_paragraph()
        run = p.add_run("Confidencial - Uso interno solamente")
        run.font.name = 'Arial'
        run.font.size = Pt(8)
        run.font.color.rgb = RGBColor(0x99, 0x99, 0x99)
        p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Guardar
        safe_name = clean_filename(compound_name)
        file_path = os.path.join(output_path, f"Informe_{safe_name}_{i+1}.docx")
        try:
            doc.save(file_path)
            success_count += 1
        except Exception as e:
            alt_path = os.path.join(output_path, f"Informe_{i+1}.docx")
            doc.save(alt_path)
            success_count += 1

    except Exception as e:
        print(f"Error procesando fila {i+1}: {str(e)}")

print(f"\n✅ Proceso completado con {success_count}/{len(df)} informes generados.")
print(f"📂 Informes guardados en: {output_path}\n")


✅ Proceso completado con 300/300 informes generados.
📂 Informes guardados en: D:ExamenParcial2ComputacionParaBioingenieria\parcial2_Machicado_Martinez_GabrielAntonio_ejercicio2\informes_BBBP

